In [23]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 5. llm을 이용하여 AI와 Interation
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# 4. 포름프트에 메모리에서 불러온 내용을 결합
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 3. load_memory 함수: 메모리에 저장되어 있는 내용을 불러옴
def load_memory(_):
    return memory.load_memory_variables({})["history"]


# 2. 체인 생성: 러너블로 메모리를 불러온 후 프롬프트에 결합 후 llm 실행
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


# 1. invoke_chain 함수: 질문을 넣어 체인을 부름
def invoke_chain(question):
    result = chain.invoke({"question": question})
    # 6. 체인 실행 결과를 메모리에 다시 넣어줌
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    # 7. 체인 실행 결과를 보여줌
    print(result)

In [24]:
invoke_chain("My name is Vero")

content='Hello Vero! How can I assist you today?'


In [25]:
invoke_chain("What is my name?")

content='Your name is Vero.'
